## 需要什么信息 都在哪里

In [1]:

import pyshark as ps
cap = ps.FileCapture("splited_00008_20120530092232.pcap")
dir (cap)
pktlist = []
for pkt in cap:
    pktlist.append(pkt)

In [8]:
'''
def get_burst(cap,burst_length):
    burst_list = []
    flow_list = []
    startTime = cap[0].sniff_timestamp
    for pkt in cap:
        flow_list.append(pkt)
        if float(pkt.sniff_timestamp)*1000-float(startTime)*1000>burst_length:
            burst_list.append(flow_list)
            startTime = pkt.sniff_timestamp
            flow_list=[]
    return burst_list
'''
def get_burst(cap,burst_length):
    burst_list = []
    flow_list = []
    startTime = cap[0].sniff_timestamp
    for pkt in cap:     
        if float(pkt.sniff_timestamp)*1000-float(startTime)*1000>burst_length:
            burst_list.append(flow_list)            
            flow_list=[]
        startTime = pkt.sniff_timestamp
        flow_list.append(pkt)
    return burst_list
# burst to flow
def burst2flow(burst):
    print("burst to flow :len burst",len(burst))
    dict_={}
    #flow_list = [[] for i in range(len(burst))]
    flow_list = []
    tmp_list=[]
    i=0            # key-value :ret-index
    for pkt in burst:
        isdsd = common_attribute(pkt)
        #print("wu yuan zu is",isdsd)
        tuple_=tuple(isdsd)
        if tuple_ in dict_ :
            #print("key alresdy exists",tuple_)
            flow_list[dict_[tuple_]].append(pkt)
            pass
        else:
            #print("key doesn't exists",tuple_)
            dict_[tuple_] = i      # new key for the new flow
            tmp_list.append(pkt)
            flow_list.append(tmp_list)
            i=i+1
            pass
        tmp_list = []         # reset
    return flow_list

# cap->flow 
def get_flow(burst_list):
    flow_ret=[]
    print("get_flow len burst_list",len(burst_list))
    for burst in burst_list:
        flow_ret.append(burst2flow(burst))
    return flow_ret

# ret ip port 
def common_attribute(pkt):   
    if pkt.__contains__("udp"):
        if pkt.__contains__("ip"):
            src = pkt.ip.src
            #print(src)
            dst = pkt.ip.dst
            srcport = pkt.udp.srcport
            dstport = pkt.udp.dstport
        else:
            src = -1 #none
            dst = -1
            srcport = -1
            dstport = -1
            pass
    elif pkt.__contains__("tcp"):
        if pkt.__contains__("ip"):
            src = pkt.ip.src
            dst = pkt.ip.dst
            srcport = pkt.tcp.srcport
            dstport = pkt.tcp.dstport
        else:
            src = -1 #none
            dst = -1
            srcport = -1
            dstport = -1
            pass
    else :
        src = -1 #none
        dst = -1
        srcport = -1
        dstport = -1
    ret = [src,srcport]
    return ret



In [9]:
xx = get_burst(cap,1000)
print(len(xx))

#print(dir(xx[0][0]))
#print(len(xx[0]))
#get_flow(xx)
#x,xx,xxx,xxxx=common_attribute(pktlist[1])
#print(x,xx,xxx,xxxx)
ret=get_flow(xx)
for j in range(len(xx)):   
    for i in range(len(ret[j])):
        print("burst ",j,"flow ",i,"is ",len(ret[j][i]),(ret[j][i]))

45
get_flow len burst_list 45
burst to flow :len burst 4
burst to flow :len burst 260
burst to flow :len burst 2
burst to flow :len burst 266
burst to flow :len burst 2
burst to flow :len burst 258
burst to flow :len burst 16
burst to flow :len burst 1048
burst to flow :len burst 259
burst to flow :len burst 262
burst to flow :len burst 258
burst to flow :len burst 3
burst to flow :len burst 2
burst to flow :len burst 271
burst to flow :len burst 2
burst to flow :len burst 266
burst to flow :len burst 2
burst to flow :len burst 261
burst to flow :len burst 2
burst to flow :len burst 8
burst to flow :len burst 527
burst to flow :len burst 258
burst to flow :len burst 16
burst to flow :len burst 3
burst to flow :len burst 3
burst to flow :len burst 3
burst to flow :len burst 266
burst to flow :len burst 2
burst to flow :len burst 258
burst to flow :len burst 270
burst to flow :len burst 2
burst to flow :len burst 5
burst to flow :len burst 613
burst to flow :len burst 1581
burst to flow 

In [72]:
pkt = pktlist[1]
print("dir pkt:\n",dir(pkt))
print("pkt.layers",pkt.layers)
print("----------------------")
print(pkt.sniff_timestamp)
print("dir pkt.layer[1]\n",dir(pkt.layers[1]))
print("dir pkt.ip\n",pkt.udp.dstport)
print(pkt.layers[1]==dir(pkt.ip))
print("----------------------")
print("dir pkt.eth:\n",dir(pkt.eth))
print(type(pkt.eth))
print("----------------------")

dir pkt:
 ['__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_packet_string', 'captured_length', 'eth', 'frame_info', 'get_multiple_layers', 'highest_layer', 'interface_captured', 'ip', 'layers', 'length', 'nbns', 'number', 'pretty_print', 'show', 'sniff_time', 'sniff_timestamp', 'transport_layer', 'udp']
pkt.layers [<ETH Layer>, <IP Layer>, <UDP Layer>, <NBNS Layer>]
----------------------
1504749101.669422000
dir pkt.layer[1]
 ['', 'DATA_LAYER', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__ini

In [61]:
print("pkt udp:",pkt.__contains__("udp"))
print("pkt udp",pkt.udp.field_names)
print("pkt udp:",dir(pkt.udp))

pkt udp: True
pkt udp ['srcport', 'dstport', 'port', 'length', 'checksum', 'checksum_status', 'stream']
pkt udp: ['DATA_LAYER', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_all_fields', '_field_prefix', '_get_all_field_lines', '_get_all_fields_with_alternates', '_get_field_or_layer_repr', '_get_field_repr', '_layer_name', '_sanitize_field_name', 'checksum', 'checksum_status', 'dstport', 'field_names', 'get', 'get_field', 'get_field_by_showname', 'get_field_value', 'layer_name', 'length', 'port', 'pretty_print', 'raw_mode', 'srcport', 'stream']


In [8]:
for pkt in pktlist:
    #print("dir pkt:\n",dir(pkt))
    print(pkt.sniff_timestamp)
    print(pkt.layers)
    #print("dir pkt.eth:\n",dir(pkt.eth))
    #print(type(pkt.eth))
    print("----------------------")


1504749101.667289000
[<ETH Layer>, <ARP Layer>]
----------------------
1504749101.669422000
[<ETH Layer>, <IP Layer>, <UDP Layer>, <NBNS Layer>]
----------------------
1504749101.670280000
[<ETH Layer>, <IP Layer>, <IGMP Layer>]
----------------------
1504749101.670915000
[<ETH Layer>, <IP Layer>, <IGMP Layer>]
----------------------
1504749101.670916000
[<ETH Layer>, <IP Layer>, <IGMP Layer>]
----------------------
1504749101.671589000
[<ETH Layer>, <IP Layer>, <UDP Layer>, <LLMNR Layer>]
----------------------
1504749101.685671000
[<ETH Layer>, <IP Layer>, <IGMP Layer>]
----------------------
1504749101.686327000
[<ETH Layer>, <IP Layer>, <UDP Layer>, <NBNS Layer>]
----------------------
1504749101.686355000
[<ETH Layer>, <IP Layer>, <UDP Layer>, <LLMNR Layer>]
----------------------
1504749101.703650000
[<ETH Layer>, <ARP Layer>]
----------------------
1504749101.779361000
[<ETH Layer>, <IP Layer>, <UDP Layer>, <LLMNR Layer>]
----------------------
1504749101.779510000
[<ETH Layer>,

1504749110.357945000
[<ETH Layer>, <IP Layer>, <TCP Layer>]
----------------------
1504749110.358961000
[<ETH Layer>, <IP Layer>, <TCP Layer>]
----------------------
1504749110.359012000
[<ETH Layer>, <IP Layer>, <TCP Layer>]
----------------------
1504749110.361032000
[<ETH Layer>, <IP Layer>, <TCP Layer>]
----------------------
1504749110.361079000
[<ETH Layer>, <IP Layer>, <TCP Layer>]
----------------------
1504749110.420385000
[<ETH Layer>, <IP Layer>, <TCP Layer>, <SSL Layer>]
----------------------
1504749110.423775000
[<ETH Layer>, <IP Layer>, <TCP Layer>, <SSL Layer>]
----------------------
1504749110.445357000
[<ETH Layer>, <IP Layer>, <TCP Layer>, <SSL Layer>]
----------------------
1504749110.449344000
[<ETH Layer>, <IP Layer>, <UDP Layer>, <NBNS Layer>]
----------------------
1504749110.620388000
[<ETH Layer>, <IP Layer>, <TCP Layer>]
----------------------
1504749110.630336000
[<ETH Layer>, <IP Layer>, <TCP Layer>]
----------------------
1504749110.645338000
[<ETH Layer>,

In [16]:
print(pkt.eth)
print(pkt.layers)


Layer ETH:
	Destination: 00:23:89:56:f7:91
	Address: 00:23:89:56:f7:91
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Source: 28:d2:44:3c:ab:ce
	Type: IPv4 (0x0800)
	Address: 28:d2:44:3c:ab:ce
	.... ..0. .... .... .... .... = LG bit: Globally unique address (factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)

[<ETH Layer>, <IP Layer>, <TCP Layer>, <SSL Layer>]
